![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [52]:
WITH company_join_all AS (
SELECT
	*,
	EXTRACT(year from dates.date_joined) as year
FROM dates , industries , companies , funding
WHERE dates.company_id = industries.company_id AND companies.company_id = industries.company_id 	AND	funding.company_id = companies.company_id)
	, top_industry AS (
SELECT
	industry,
	COUNT(company) AS num_unicorns,
	RANK() OVER (ORDER BY COUNT(company) DESC) AS ranking
FROM company_join_all
WHERE year BETWEEN 2019 AND 2021
GROUP BY industry)

SELECT
	industry,
	year,
	COUNT(company) AS num_unicorns,
	ROUND(AVG(valuation)/1000000000,2) AS average_valuation_billions
FROM company_join_all 
WHERE industry IN (SELECT industry FROM top_industry WHERE ranking BETWEEN 1 AND 3)
	AND year BETWEEN 2019 AND 2021
GROUP BY industry , year
ORDER BY year , num_unicorns;



,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2019,12,2.58
1,Internet software & services,2019,13,4.23
2,Fintech,2019,20,6.80
3,Fintech,2020,15,4.33
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Internet software & services,2020,20,4.35
6,E-commerce & direct-to-consumer,2021,47,2.47
7,Internet software & services,2021,119,2.15
8,Fintech,2021,138,2.75
